# Example figure plotting

This notebook creates the example figures stored in `doc/figures` and shown in the `README.md` file.

In [ ]:
import xarray as xr
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path

sns.set(rc={'figure.figsize':(15,15)})

paths = Path("./results/ssp2-2.6/")
paths = list(paths.rglob('Europe.nc'))

In [ ]:
fig, axes = plt.subplots(ncols=3, nrows=3, sharey='row')
axes = [ax for l in axes for ax in l]

for (path, ax) in zip(paths, axes):

    year = path.parts[2]
    era_year = path.parts[3]

    data = xr.open_dataarray(path)
    data = data.sel(region_code='DE')

    df = data.to_dataframe().reset_index()
    sns.violinplot(x=df['region_code'],y=df['Electricity demand']
                  #, label=f'Year: {year} | {era_year}'
                  , ax=ax)
    
    ax.set_title(f'Year: {year} | {era_year}')
    ax.set_xlabel('')
    ax.set_ylabel('Electricity demand [MWh]')
    
plt.savefig('doc/figures/DE_violin-plot_multi-years_multi-erayears.png', dpi=150)

In [ ]:
fig, axes = plt.subplots(ncols=1, nrows=3, sharey='row')
axes = [e for l in [[ax]*3 for ax in axes] for e in l]

for (path, ax) in zip(paths, axes):

    year = path.parts[2]
    era_year = path.parts[3]

    data = xr.open_dataarray(path)
    data = data.sel(region_code='DE')
    
    data.groupby('time.weekofyear').sum().plot(ax=ax, label=f'{era_year}')
    
    ax.set_title(f'SSP2-2.6 for {year}')
    ax.legend()
    ax.set_ylabel('Electricity demand [MWh]')
    
plt.savefig('doc/figures/DE_weekly-demand_multi-years_multi-erayears.png', dpi=150)

In [ ]:
fig, axes = plt.subplots(ncols=1, nrows=3, sharey='row')
axes = [e for l in [[ax]*3 for ax in axes] for e in l]

for (path, ax) in zip(paths, axes):
    
    year = path.parts[2]
    era_year = path.parts[3]
    
    data = xr.open_dataarray(path)
    data = data.sel(region_code='DE')
    
    time = pd.date_range(f"{year}",periods=8760,freq='H')
    data = data.assign_coords({'time':time})
        
    data = data.isel(time=slice(900,1048))
    
    data.plot(ax=ax, label=f'{era_year}')
    
    ax.set_title(f'SSP2-2.6 for {year}')
    ax.legend()
    ax.set_ylabel('Electricity demand [MWh]')
    ax.set_xlabel('')
    
plt.savefig('doc/figures/DE_hourly-demand-excerpt_multi-years_multi-erayears.png', dpi=150)

In [ ]:
import geopandas as gpd
import numpy as np 
import pycountry as pyc
import matplotlib as mpl

data = None
data = xr.open_mfdataset("results/ssp2-2.6/2030/era5_2013/*.nc", combine="nested")

data = data.sum(dim='time')

# GADM data; manually download and extract
# Source: https://biogeo.ucdavis.edu/data/gadm3.6/gadm36_gpkg.zip
world = gpd.read_file('../resources/gadm36_levels.gpkg')

plot_data = pd.merge(
    left=world,
    right=data.to_dataframe(),
    left_on='NAME_0',
    right_on='region_name',
    how='inner')

plot_data.loc[plot_data['Electricity demand'] == 0., 'Electricity demand'] = np.NaN

fig, ax = plt.subplots(1, 1, figsize=(16,10))

plot_data.plot(column='Electricity demand',cmap='viridis',
        norm=mpl.colors.LogNorm(vmin=1, vmax=plot_data['Electricity demand'].max()),
        missing_kwds=dict(color='grey'),
        ax=ax,
        legend=True,
        legend_kwds={'label': "Demand [MWh] (missing countries in gray)", 'orientation': "horizontal"})

fig.suptitle("Annual electricity demand 2030 (SSP2, RCP2.6)")
fig.tight_layout()

plt.savefig('doc/figures/world_annual-demand_2030.png', dpi=150)